In [2]:
import torch 

import torch.nn as nn

import torch.nn.functional as F

import math

import numpy as np

import pandas as pd

import os 

import random

import time

#### Implementing LORA Using PEFT  and open source github tag generator using T5 small 

what is T5 ?


It is NLP model for text to text problem , meaning both the input and output are text strings 




In [2]:
### LORA is Low Rank Adaptation  is a parameter efficient fine tuning method or technique for large language models (LLMs) that aims to reduce the number of trainable parameters while maintaining performance.

In [3]:
# from google.colab import userdata
# import os

# os.environ["HUGGINGFACE_TOKEN"] = userdata.get("HUGGINGFACE_TOKEN")

In [ ]:
# !pip install transformers datasets peft bitsandbytes -q


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import os
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig

In [ ]:
# In a notebook cell:
# !pip install --upgrade transformers accelerate

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     -------------------------------------  41.0/42.0 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 689.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.5/11.3 MB 46.9 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.3 MB 39.9 MB/s eta 0:00:01
   ------------------ --------------------- 5.3/11.3 MB 42.4 MB/s eta 0:00:01
   --------------------------- ------------ 7.8/11.3 MB 41.8 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.3 MB 43.7 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 46.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/374.7 kB ? eta -:--:--
   ---------------------------------------- 374.7/374.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/561.5 kB ? eta -:--:--
   ----------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.3.5 requires transformers<4.37.0,>=3.4.0, but you have transformers 4.55.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kavitha padala\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
# Load existing dataset with only a "train" split
from datasets import  DatasetDict

dataset = load_dataset("zamal/github-meta-data")  # returns DatasetDict

# Split the train set into train and validation
split = dataset["train"].train_test_split(test_size=0.1, seed=42)

# Wrap into a new DatasetDict
dataset_dict = DatasetDict({"train": split["train"], "validation": split["test"]})

README.md:   0%|          | 0.00/473 [00:00<?, ?B/s]

c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kavitha padala\.cache\huggingface\hub\datasets--zamal--github-meta-data. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not i

train-00000-of-00001.parquet:   0%|          | 0.00/32.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/614 [00:00<?, ? examples/s]

In [6]:
dataset_dict['train'][0]

{'input': 'typescript css library postcss tailwindcss',
 'target': 'tailwindcss, css, css-framework, postcss, functional-css, utility-classes, responsive'}

In [7]:
print(len(dataset_dict["train"]))
print(len(dataset_dict["validation"]))

552
62


In [8]:
from transformers import AutoTokenizer

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
def preprocess(batch):
    inputs = batch["input"]
    targets = batch["target"]
    model_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    labels = tokenizer(targets, return_tensors="pt", padding=True, truncation=True).input_ids
    model_inputs["labels"] = labels
    return model_inputs


In [10]:
tokenized = dataset_dict.map(preprocess, batched=True, remove_columns=dataset_dict["train"].column_names)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [11]:
tokenized["train"][0]

{'input_ids': tensor([  686, 11815,     3,    75,     7,     7,  3595,   442,    75,     7,
             7,  9891,  5165,    75,     7,     7,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([ 9891,  5

In [12]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],  # Adjust based on model architecture
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

In [14]:
model = get_peft_model(model, lora_config)

In [18]:
training_args = TrainingArguments(
    output_dir="./t5_tag_generator",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    num_train_epochs=25,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    push_to_hub=True,
    hub_model_id="zamal/github-tag-generatorr",  # Replace with your Hugging Face username
    hub_token=os.environ["HUGGINGFACE_TOKEN"],
)

In [19]:
from transformers import Trainer
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Kavitha padala\AppData\Local\Temp\ipykernel_19840\2744959247.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\data\data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
50,8.405300,7.104400
100,4.423500,3.334789
150,2.811000,2.331543
200,2.509200,2.101823
250,2.183000,1.991302
300,1.935900,1.901142
350,1.739500,1.805873
400,1.658900,1.712887
450,1.588900,1.639096
500,1.652300,1.581841


TrainOutput(global_step=1725, training_loss=1.8801535841347514, metrics={'train_runtime': 148.5452, 'train_samples_per_second': 92.901, 'train_steps_per_second': 11.613, 'total_flos': 303131197440000.0, 'train_loss': 1.8801535841347514, 'epoch': 25.0})

In [21]:
import torch


def generate_tags(query, model, tokenizer, max_length=64, num_beams=5):
    model.eval()
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(
        model.device
    )

    with torch.no_grad():
        output = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            decoder_start_token_id=tokenizer.pad_token_id,  # 👈 required for T5
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [23]:
generate_tags("looking for repositories on image augmentation no code implementations,html and css and js ", model, tokenizer)

'image augmentation, no code implementations, css, js, js, css, css, js'

In [24]:
# Save model, tokenizer, and config to local output directory
model_path = "../models/t5_tag_generator"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print("✅ Model and tokenizer saved locally at:", model_path)

✅ Model and tokenizer saved locally at: ../models/t5_tag_generator


In [25]:
inputs = [
    "Need an AI tool to convert customer voice calls into structured CRM record",
    "How to train a text summarization model using Pegasus or BART",
    "Fine-tuning BERT for spam detection in emails",
]

for text in inputs:
    print(f"📥 Input: {text}")
    print(f"🏷️ Tags: {generate_tags(text, model, tokenizer)}\n")

📥 Input: Need an AI tool to convert customer voice calls into structured CRM record
🏷️ Tags: voice-calls, CRM-records, voice-calls

📥 Input: How to train a text summarization model using Pegasus or BART
🏷️ Tags: text summarization, pegasus, bART, et al.

📥 Input: Fine-tuning BERT for spam detection in emails
🏷️ Tags: BERT, spam-detecting, email-tuning, email-tuning, email-tuning



In [26]:
def generate_tags(text, model, tokenizer, max_length=64, num_beams=5):
    input_text = text
    inputs = tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(
        model.device
    )

    with torch.no_grad():
        output = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            decoder_start_token_id=tokenizer.pad_token_id,
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Deduplicate and clean tags
    tags = [t.strip().lower() for t in decoded.split(",")]
    unique_tags = list(dict.fromkeys(tags))  # preserve order + remove duplicates
    return ", ".join(unique_tags)

In [27]:
inputs = [
    "Need an AI tool to convert customer voice calls into structured CRM record",
    "How to train a text summarization model using Pegasus or BART",
    "Fine-tuning BERT for spam detection in emails",
]

for text in inputs:
    print(f"📥 Input: {text}")
    print(f"🏷️ Tags: {generate_tags(text, model, tokenizer)}\n")

📥 Input: Need an AI tool to convert customer voice calls into structured CRM record
🏷️ Tags: voice-calls, crm-records

📥 Input: How to train a text summarization model using Pegasus or BART
🏷️ Tags: text summarization, pegasus, bart, et al.

📥 Input: Fine-tuning BERT for spam detection in emails
🏷️ Tags: bert, spam-detecting, email-tuning

